In [1]:
!pip -q install ftfy regex tqdm
!pip -q install git+https://github.com/openai/CLIP.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.6 MB/s eta 0:00:

In [2]:
#train-val-test: 70:15:15
!gdown 1u9XcSIB4-UMmgSGa_or2B3bBbgiNZe5I
!unzip -q data_cifar10_ent_split.zip

Downloading...
From (original): https://drive.google.com/uc?id=1u9XcSIB4-UMmgSGa_or2B3bBbgiNZe5I
From (redirected): https://drive.google.com/uc?id=1u9XcSIB4-UMmgSGa_or2B3bBbgiNZe5I&confirm=t&uuid=5a9aa61d-ccdf-4c2b-9282-464d1eca29e0
To: /kaggle/working/data_cifar10_ent_split.zip
100%|██████████████████████████████████████| 1.06G/1.06G [00:11<00:00, 89.6MB/s]


In [3]:

import os
import torch
import clip
from PIL import Image
import torch.nn.functional as F
import random
from sklearn.model_selection import train_test_split
import numpy as np

import torch.nn as nn
from random import shuffle
from tqdm import tqdm
import torch.nn.init as init

from torch.optim.lr_scheduler import ReduceLROnPlateau

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 166MiB/s]


In [7]:
# --- Load images & labels ---
def load_images_and_labels(root_dir):
    embeddings, labels, paths = [], [], []
    class_names = sorted(os.listdir(root_dir))
    class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}
    idx_to_class = {v: k for k, v in class_to_idx.items()}

    for cls in tqdm(class_names, desc="Loading images"):
        cls_path = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_path): continue
        for fname in os.listdir(cls_path):
            img_path = os.path.join(cls_path, fname)
            try:
                image = preprocess(Image.open(img_path).convert("RGB")).unsqueeze(0).to(device)
                with torch.no_grad():
                    emb = model.encode_image(image)
                    emb = emb / emb.norm(dim=-1, keepdim=True)  # Normalize
                    embeddings.append(emb.squeeze(0))
                    labels.append(class_to_idx[cls])
                    paths.append(img_path)
            except Exception as e:
                print(f"❌ Error loading {img_path}: {e}")

    return torch.stack(embeddings), torch.tensor(labels), paths, class_to_idx, idx_to_class

# --- Load test data ---
embeddings, labels, paths, class_to_idx, idx_to_class = load_images_and_labels("./test")
labels = labels.to(device)

print(f"✅ Loaded {len(embeddings)} images from test set ({len(class_to_idx)} classes)")


Loading images: 100%|██████████| 7/7 [00:05<00:00,  1.31it/s]

✅ Loaded 284 images from test set (7 classes)


In [8]:
# --- Cosine similarity matrix ---
similarity_matrix = embeddings @ embeddings.T
similarity_matrix.fill_diagonal_(-float('inf'))  # exclude self-match

def compute_recall_at_k_by_class(sim_matrix, labels, idx_to_class, ks=[1, 5, 10]):
    recalls = {}
    for k in ks:
        correct = 0
        for i in range(len(labels)):
            true_label = labels[i].item()

            topk = sim_matrix[i].topk(k, dim=0).indices
            topk_labels = [labels[j].item() for j in topk]

            if true_label in topk_labels:
                correct += 1

        recalls[k] = correct / len(labels)
    return recalls


recalls = compute_recall_at_k_by_class(similarity_matrix, labels, idx_to_class)
for k, v in recalls.items():
    print(f"🎯 Recall@{k:<2} (by class): {v:.4f}")


🎯 Recall@1  (by class): 0.4930
🎯 Recall@5  (by class): 0.8803
🎯 Recall@10 (by class): 0.9683


In [9]:
def compute_recall_at_k_per_class(sim_matrix, labels, idx_to_class, ks=[1, 5, 10]):
    class_recalls = {k: {} for k in ks}
    class_counts = {}

    unique_labels = torch.unique(labels)

    for cls_idx in unique_labels:
        cls_mask = (labels == cls_idx)
        cls_indices = cls_mask.nonzero(as_tuple=True)[0]
        class_counts[cls_idx.item()] = len(cls_indices)

        for k in ks:
            correct = 0
            for i in cls_indices:
                topk = sim_matrix[i].topk(k=k).indices
                topk_labels = labels[topk]
                if cls_idx.item() in topk_labels.tolist():
                    correct += 1

            recall = correct / len(cls_indices) if len(cls_indices) > 0 else 0.0
            class_name = idx_to_class[cls_idx.item()]
            class_recalls[k][class_name] = recall

    # Tính macro average cho mỗi k
    macro_avg = {
        k: sum(class_recalls[k].values()) / len(class_recalls[k])
        for k in ks
    }

    return class_recalls, macro_avg
class_recalls, macro_avg = compute_recall_at_k_per_class(similarity_matrix, labels, idx_to_class)

print("\n🎯 Recall@K theo từng class:")
for k in [1, 5, 10]:
    print(f"\n🔹 Recall@{k}")
    for cls, score in class_recalls[k].items():
        print(f"  {cls:12}: {score:.4f}")
    print(f"👉 Macro Avg Recall@{k}: {macro_avg[k]:.4f}")



🎯 Recall@K theo từng class:

🔹 Recall@1
  ear-left    : 0.5714
  ear-right   : 0.6333
  nose-left   : 0.5352
  nose-right  : 0.3333
  throat      : 0.4706
  vc-closed   : 0.4688
  vc-open     : 0.5676
👉 Macro Avg Recall@1: 0.5115

🔹 Recall@5
  ear-left    : 0.8929
  ear-right   : 0.9333
  nose-left   : 0.9577
  nose-right  : 0.8116
  throat      : 0.6471
  vc-closed   : 0.8438
  vc-open     : 0.9459
👉 Macro Avg Recall@5: 0.8618

🔹 Recall@10
  ear-left    : 0.9643
  ear-right   : 0.9667
  nose-left   : 1.0000
  nose-right  : 0.9420
  throat      : 0.8824
  vc-closed   : 0.9688
  vc-open     : 1.0000
👉 Macro Avg Recall@10: 0.9606
